both predefined and userinput done

In [3]:
!pip install openai


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


**Basic foundation for task 1**

In [ ]:
import openai
import os

# Set up Groq API client (OpenAI-compatible)
os.environ["GROQ_API_KEY"] = ""  
client = openai.OpenAI(
    base_url="https://api.groq.com/openai/v1",
    api_key=os.environ.get("GROQ_API_KEY")
)

# Model to use (free tier available on Groq, fast and capable)
MODEL = "openai/gpt-oss-20b"

# Function to get chat completion from Groq API
def get_completion(messages, model=MODEL):
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0.7,
        max_tokens=500
    )
    return response.choices[0].message.content

# Function to summarize conversation history
def summarize_history(history):
    # Format history for summarization prompt
    formatted_history = "\n".join([f"{msg['role'].capitalize()}: {msg['content']}" for msg in history])
    prompt = [
        {"role": "system", "content": "You are a helpful assistant that summarizes conversations concisely, capturing key points, decisions, and context without losing important details."},
        {"role": "user", "content": f"Summarize the following conversation in a concise paragraph:\n\n{formatted_history}"}
    ]
    summary = get_completion(prompt)
    return summary

# Class for managing conversation with truncation and periodic summarization
class ConversationManager:
    def __init__(self, max_turns=None, max_chars=None, max_words=None, summarize_every_k=None):
        """
        Initialize conversation manager.
        - max_turns: Limit to last n messages (turns).
        - max_chars: Max total characters in history.
        - max_words: Max total words in history.
        - summarize_every_k: Summarize and replace history every k-th interaction.
        """
        self.history = []
        self.max_turns = max_turns
        self.max_chars = max_chars
        self.max_words = max_words
        self.summarize_every_k = summarize_every_k
        self.interaction_count = 0

    def truncate_history(self):
        """Apply truncation based on settings."""
        if self.max_turns:
            self.history = self.history[-self.max_turns:]

        # Concatenate contents for length checks
        full_content = " ".join([msg['content'] for msg in self.history])

        if self.max_chars:
            while len(full_content) > self.max_chars and len(self.history) > 1:
                removed = self.history.pop(0)
                full_content = " ".join([msg['content'] for msg in self.history])

        if self.max_words:
            while len(full_content.split()) > self.max_words and len(self.history) > 1:
                removed = self.history.pop(0)
                full_content = " ".join([msg['content'] for msg in self.history])

    def check_periodic_summarization(self):
        """Summarize and replace history if at k-th interaction."""
        if self.summarize_every_k and self.interaction_count % self.summarize_every_k == 0 and self.interaction_count > 0:
            if len(self.history) > 1:  # Only summarize if there's history
                summary = summarize_history(self.history)
                print(f"\n--- Periodic Summarization (after {self.interaction_count} interactions) ---\nSummary: {summary}\n")
                self.history = [{"role": "system", "content": f"Summary of previous conversation: {summary}"}]

    def chat(self, user_input, system_prompt="You are a helpful assistant."):
        """Handle a user message, get response, update history, apply truncation and summarization."""
        self.interaction_count += 1

        # Build messages: system prompt + history + user input
        messages = [{"role": "system", "content": system_prompt}] + self.history + [{"role": "user", "content": user_input}]

        # Get assistant response
        response = get_completion(messages)

        # Append user and assistant to history
        self.history.append({"role": "user", "content": user_input})
        self.history.append({"role": "assistant", "content": response})

        # Apply truncation
        self.truncate_history()

        # Check for periodic summarization
        self.check_periodic_summarization()

        return response

# Demonstration
print("=== Demonstration: Basic Conversation with No Truncation/Summarization ===")
manager = ConversationManager()
samples = [
    "Hello, what's the weather like today?",
    "Tell me a joke.",
    "What's the capital of France?",
    "Explain quantum computing simply."
]
for sample in samples:
    response = manager.chat(sample)
    print(f"User: {sample}\nAssistant: {response}\n")
print(f"Final History: {manager.history}\n")

print("=== Demonstration: Truncation by Max Turns (last 4 messages) ===")
manager_turns = ConversationManager(max_turns=4)  # Last 2 turns (4 messages)
for sample in samples:
    response = manager_turns.chat(sample)
    print(f"User: {sample}\nAssistant: {response}\n")
print(f"Final History (truncated to last 4 messages): {manager_turns.history}\n")

print("=== Demonstration: Truncation by Max Characters (500 chars total) ===")
manager_chars = ConversationManager(max_chars=500)
for sample in samples:
    response = manager_chars.chat(sample)
    print(f"User: {sample}\nAssistant: {response}\n")
print(f"Final History (truncated by chars): {manager_chars.history}\n")

print("=== Demonstration: Truncation by Max Words (100 words total) ===")
manager_words = ConversationManager(max_words=100)
for sample in samples:
    response = manager_words.chat(sample)
    print(f"User: {sample}\nAssistant: {response}\n")
print(f"Final History (truncated by words): {manager_words.history}\n")

print("=== Demonstration: Periodic Summarization Every 3rd Run ===")
manager_periodic = ConversationManager(summarize_every_k=3)
for i, sample in enumerate(samples, 1):
    response = manager_periodic.chat(sample)
    print(f"Interaction {i} - User: {sample}\nAssistant: {response}\n")
    print(f"History after interaction {i}: {manager_periodic.history}\n")

=== Demonstration: Basic Conversation with No Truncation/Summarization ===
User: Hello, what's the weather like today?
Assistant: I’m sorry, but I don’t have access to real‑time weather data.  
You can quickly check the current conditions by:

- Using a weather app on your phone (e.g., Weather.com, AccuWeather, or the built‑in weather app).  
- Visiting a weather website such as [weather.com](https://weather.com) or [accuweather.com](https://www.accuweather.com).  
- Asking a voice assistant like Siri, Google Assistant, or Alexa.

If you tell me your city or zip code, I can give you a general idea of the typical weather for this time of year!

User: Tell me a joke.
Assistant: Sure thing! Here’s a light‑hearted one for you:

**Why don’t skeletons fight each other?**

Because they don’t have the guts! 😄

User: What's the capital of France?
Assistant: The capital of France is **Paris**.

User: Explain quantum computing simply.
Assistant: **Quantum computing in a nutshell**

| Classical co

**Base task 2**

In [ ]:
# Task 2: JSON Schema Classification & Information Extraction
import json
import re

# Define JSON schema (updated: email, phone, age optional to allow null)
json_schema = {
    "name": "pii_extraction",
    "schema": {
        "type": "object",
        "properties": {
            "name": {
                "type": "string",
                "description": "Full name of the person (e.g., 'John Doe'). Required."
            },
            "email": {
                "type": ["string", "null"],
                "description": "Email address (e.g., 'john@example.com'). Null if not present.",
                "pattern": "^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\\.[a-zA-Z]{2,}$"
            },
            "phone": {
                "type": ["string", "null"],
                "description": "Phone number (e.g., '+1-123-456-7890'). Null if not present.",
                "pattern": "^[+]?[0-9\\s\\-\\(\\)]{10,}$"
            },
            "location": {
                "type": "string",
                "description": "Location or city (e.g., 'New York, NY'). Required."
            },
            "age": {
                "type": ["integer", "null"],
                "description": "Age as a number (e.g., 30). Null if not stated."
            }
        },
        "required": ["name", "location"],  # Only name and location required
        "additionalProperties": False
    }
}

# Define tool for function calling
tools = [
    {
        "type": "function",
        "function": {
            "name": "extract_pii",
            "description": "Extract PII details from a chat transcript.",
            "parameters": json_schema["schema"]
        }
    }
]

# Function to extract PII
def extract_pii_from_chat(chat_transcript, model=MODEL):
    messages = [
        {"role": "system", "content": "Extract structured PII from chat transcripts. Only extract explicitly mentioned details; use null for missing email, phone, or age."},
        {"role": "user", "content": f"Extract PII from this chat: {chat_transcript}"}
    ]
    
    try:
        response = client.chat.completions.create(
            model=model,
            messages=messages,
            tools=tools,
            tool_choice="auto",
            temperature=0.1,
            max_tokens=500
        )
        
        if response.choices[0].message.tool_calls:
            tool_call = response.choices[0].message.tool_calls[0]
            extracted_data = json.loads(tool_call.function.arguments)
            return extracted_data
        else:
            return {"error": "No tool call triggered"}
    except Exception as e:
        return {"error": f"API error: {str(e)}"}

# Validation function
def validate_extraction(extracted_data, schema=json_schema["schema"]):
    validation_results = {
        "valid": True,
        "details": {}
    }
    
    required_keys = schema["required"]
    for key in required_keys:
        if key not in extracted_data:
            validation_results["valid"] = False
            validation_results["details"][key] = "Missing required key"
            continue
        
        value = extracted_data[key]
        expected_types = schema["properties"][key]["type"]
        expected_types = expected_types if isinstance(expected_types, list) else [expected_types]
        
        if value is not None and not any(isinstance(value, type_map[t]) for t in expected_types):
            validation_results["valid"] = False
            validation_results["details"][key] = f"Expected {expected_types}, got {type(value)}"
        
        if value is not None and key in ["email", "phone"] and "pattern" in schema["properties"][key]:
            pattern = schema["properties"][key]["pattern"]
            if not re.match(pattern, str(value)):
                validation_results["valid"] = False
                validation_results["details"][key] = "Does not match expected pattern"
    
    # Check optional fields (email, phone, age)
    for key in ["email", "phone", "age"]:
        if key in extracted_data and extracted_data[key] is not None:
            expected_types = schema["properties"][key]["type"]
            expected_types = expected_types if isinstance(expected_types, list) else [expected_types]
            if not any(isinstance(extracted_data[key], type_map[t]) for t in expected_types):
                validation_results["valid"] = False
                validation_results["details"][key] = f"Expected {expected_types}, got {type(extracted_data[key])}"
    
    # Check for extra properties
    if set(extracted_data.keys()) != set(schema["properties"].keys()):
        extra = set(extracted_data.keys()) - set(schema["properties"].keys())
        validation_results["valid"] = False
        validation_results["details"]["extra_properties"] = list(extra)
    
    return validation_results

# Type mapping for validation
type_map = {
    "string": str,
    "integer": int,
    "null": type(None)
}

# Demonstration: 3 sample chats
sample_chats = [
    "User: Hi, I'm John Doe, emailing from john.doe@example.com. My phone is +1-555-123-4567. I live in New York, NY, and I'm 28 years old.",
    "Assistant: Thanks for reaching out. User: Sure, my name is Jane Smith, email jane@work.com, phone 123-456-7890, and location is London, UK. Age not mentioned.",
    "User: Hello, this is Alice Johnson from Seattle, WA. No email or phone shared, and I won't tell my age."
]

print("=== Demonstration: JSON Schema PII Extraction from Sample Chats ===")
for i, chat in enumerate(sample_chats, 1):
    print(f"\n--- Sample Chat {i} ---")
    print(f"Chat Transcript:\n{chat}\n")
    
    extracted = extract_pii_from_chat(chat)
    print(f"Extracted JSON:\n{json.dumps(extracted, indent=2)}\n")
    
    validation = validate_extraction(extracted)
    print(f"Validation Results:\n{json.dumps(validation, indent=2)}\n")
    
    if validation["valid"]:
        print("✅ Valid extraction against schema.")
    else:
        print("❌ Invalid: See details above.")

=== Demonstration: JSON Schema PII Extraction from Sample Chats ===

--- Sample Chat 1 ---
Chat Transcript:
User: Hi, I'm John Doe, emailing from john.doe@example.com. My phone is +1-555-123-4567. I live in New York, NY, and I'm 28 years old.

Extracted JSON:
{
  "age": 28,
  "email": "john.doe@example.com",
  "location": "New York, NY",
  "name": "John Doe",
  "phone": "+1-555-123-4567"
}

Validation Results:
{
  "valid": true,
  "details": {}
}

✅ Valid extraction against schema.

--- Sample Chat 2 ---
Chat Transcript:
Assistant: Thanks for reaching out. User: Sure, my name is Jane Smith, email jane@work.com, phone 123-456-7890, and location is London, UK. Age not mentioned.

Extracted JSON:
{
  "error": "API error: Error code: 400 - {'error': {'message': 'Tool call validation failed: tool call validation failed: parameters for tool extract_pii did not match schema: errors: [`/age`: expected integer, but got null]', 'type': 'invalid_request_error', 'code': 'tool_use_failed', 'failed_

**Task 1 final with Markdown handelling**

In [ ]:
import openai
import os
from IPython.display import display, Markdown

# Set up Groq API client
os.environ["GROQ_API_KEY"] = ""  # Replace with your key
client = openai.OpenAI(
    base_url="https://api.groq.com/openai/v1",
    api_key=os.environ.get("GROQ_API_KEY")
)

MODEL = "openai/gpt-oss-20b"

# Internal completion (no display for scoring/chunks)
def get_completion_internal(messages, model=MODEL):
    system_prompt = messages[0]["content"] if messages and messages[0]["role"] == "system" else "You are a helpful assistant."
    messages = [{"role": "system", "content": system_prompt}] + (messages[1:] if messages[0]["role"] != "system" else messages)
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0.7,
        max_tokens=1500
    )
    return response.choices[0].message.content

# User-visible completion (markdown display)
def get_completion(messages, model=MODEL):
    content = get_completion_internal(messages, model)
    display(Markdown(content))
    return content

# Hierarchical summarization
def summarize_history(history, chunk_size=3):
    if len(history) <= chunk_size:
        formatted = "\n".join([f"{msg['role'].capitalize()}: {msg['content']}" for msg in history])
        prompt = [
            {"role": "system", "content": "Summarize this conversation concisely in markdown, using tables or lists for clarity, capturing key points and context."},
            {"role": "user", "content": formatted}
        ]
        return get_completion(prompt)
    
    chunks = [history[i:i+chunk_size] for i in range(0, len(history), chunk_size)]
    chunk_summaries = []
    for i, chunk in enumerate(chunks):
        formatted_chunk = "\n".join([f"{msg['role'].capitalize()}: {msg['content']}" for msg in chunk])
        prompt = [
            {"role": "system", "content": f"Summarize chunk {i+1} concisely."},
            {"role": "user", "content": formatted_chunk}
        ]
        chunk_summaries.append(get_completion_internal(prompt))
    
    meta_formatted = "\n".join(chunk_summaries)
    meta_prompt = [
        {"role": "system", "content": "Create a high-level markdown summary from these chunk summaries, using tables or lists, focusing on conversation flow and key insights."},
        {"role": "user", "content": meta_formatted}
    ]
    return get_completion(meta_prompt)

# Semantic relevance scorer
def score_relevance(message, current_context):
    prompt = [
        {"role": "system", "content": "Score the relevance of this message to the ongoing conversation (0-10, higher=more relevant). Respond with just the number."},
        {"role": "user", "content": f"Context: {current_context}\nMessage: {message['content']}"}
    ]
    score_str = get_completion_internal(prompt).strip()
    try:
        return int(score_str)
    except:
        return 5

# Advanced ConversationManager
class ConversationManager:
    def __init__(self, max_turns=None, max_chars=None, max_words=None, summarize_every_k=None, semantic_threshold=None, min_messages=2):
        """
        Advanced manager with:
        - max_turns: Limit to last n messages.
        - max_chars/max_words: Total length limits.
        - summarize_every_k: Hierarchical summarization every k interactions.
        - semantic_threshold: Remove messages with relevance < threshold (if set).
        - min_messages: Ensure at least this many messages remain after truncation.
        """
        self.history = []
        self.max_turns = max_turns
        self.max_chars = max_chars
        self.max_words = max_words
        self.summarize_every_k = summarize_every_k
        self.semantic_threshold = semantic_threshold
        self.min_messages = max(min_messages, 2)  # Ensure at least 2
        self.interaction_count = 0

    def apply_semantic_truncation(self):
        if len(self.history) < self.min_messages:
            return
        recent_history = self.history[-5:]  # Cap at 5 for efficiency
        current_context = self.history[-1]["content"][:200]
        scored_history = []
        for msg in recent_history:
            score = score_relevance(msg, current_context)
            if score >= self.semantic_threshold:
                scored_history.append(msg)
        # Ensure min_messages
        self.history = self.history[:max(0, len(self.history) - len(scored_history))] + scored_history
        if len(self.history) < self.min_messages and len(scored_history) > 0:
            self.history = scored_history[-self.min_messages:]

    def truncate_history(self):
        if self.semantic_threshold is not None:
            self.apply_semantic_truncation()
        
        if self.max_turns and len(self.history) > max(self.max_turns, self.min_messages):
            self.history = self.history[-(max(self.max_turns, self.min_messages)):]

        full_content = " ".join([msg['content'] for msg in self.history])
        
        if self.max_chars:
            while len(full_content) > self.max_chars and len(self.history) > self.min_messages:
                self.history.pop(0)
                full_content = " ".join([msg['content'] for msg in self.history])

        if self.max_words:
            while len(full_content.split()) > self.max_words and len(self.history) > self.min_messages:
                self.history.pop(0)
                full_content = " ".join([msg['content'] for msg in self.history])

    def check_periodic_summarization(self):
        if self.summarize_every_k and self.interaction_count % self.summarize_every_k == 0 and self.interaction_count > 0:
            if len(self.history) > 1:
                summary = summarize_history(self.history)
                print(f"\n--- Advanced Hierarchical Summarization (after {self.interaction_count} interactions) ---\n")
                self.history = [{"role": "system", "content": f"Hierarchical summary of previous conversation:\n{summary}"}]

    def chat(self, user_input, system_prompt="You are a helpful assistant. Respond using markdown for clarity (e.g., tables, lists, bold)."):
        self.interaction_count += 1
        messages = [{"role": "system", "content": system_prompt}] + self.history + [{"role": "user", "content": user_input}]
        response = get_completion(messages)
        self.history.append({"role": "user", "content": user_input})
        self.history.append({"role": "assistant", "content": response})
        self.truncate_history()
        self.check_periodic_summarization()
        return response

# Demonstrations
samples = [
    "Hello, what's the weather like today?",
    "Tell me a joke.",
    "What's the capital of France?",
    "Explain quantum computing simply."
]

print("=== Demo 1: Basic Conversation (No Truncation/Semantic) ===")
manager = ConversationManager()
for sample in samples:
    print(f"\nUser: {sample}")
    response = manager.chat(sample)
    print(f"History Length: {len(manager.history)}")
print("\nFinal History:")
for msg in manager.history:
    print(f"{msg['role'].capitalize()}: {msg['content'][:50]}...")

print("\n=== Demo 2: Semantic Truncation (Threshold=4, Max Turns=4) ===")
manager_sem = ConversationManager(max_turns=4, semantic_threshold=4)
for sample in samples:
    print(f"\nUser: {sample}")
    response = manager_sem.chat(sample)
    print(f"History Length: {len(manager_sem.history)}")
print("\nFinal History:")
for msg in manager_sem.history:
    print(f"{msg['role'].capitalize()}: {msg['content'][:50]}...")

print("\n=== Demo 3: Length Truncation (Max Chars=500, Words=100) ===")
manager_len = ConversationManager(max_chars=500, max_words=100, min_messages=2)
for sample in samples:
    print(f"\nUser: {sample}")
    response = manager_len.chat(sample)
    print(f"History Length: {len(manager_len.history)}")
print("\nFinal History:")
for msg in manager_len.history:
    print(f"{msg['role'].capitalize()}: {msg['content'][:50]}...")

print("\n=== Demo 4: Periodic Summarization (Every 3rd Run) ===")
manager_periodic = ConversationManager(summarize_every_k=3)
for i, sample in enumerate(samples, 1):
    print(f"\nInteraction {i} - User: {sample}")
    response = manager_periodic.chat(sample)
    print(f"History Length: {len(manager_periodic.history)}")
print("\nFinal History:")
for msg in manager_periodic.history:
    print(f"{msg['role'].capitalize()}: {msg['content'][:50]}...")

=== Demo 1: Basic Conversation (No Truncation/Semantic) ===

User: Hello, what's the weather like today?


I’m sorry, but I don’t have real‑time access to weather data.  
To get the most accurate forecast for today, you could:

| Option | How to use it |
|--------|---------------|
| **Weather apps** (e.g., Weather.com, AccuWeather, Weather Underground) | Open the app or website, enter your location, and view the hourly forecast. |
| **Smartphone assistants** (Siri, Google Assistant, Alexa) | Say “What’s the weather today?” and they’ll read the latest forecast. |
| **Local news websites** | Many local stations provide up‑to‑date weather updates on their sites. |
| **National weather services** (e.g., NOAA in the U.S., Met Office in the U.K.) | Visit the official site for detailed radar and alerts. |

If you let me know your city or ZIP code, I can give you a general idea of the typical weather pattern for this time of year, but for the exact current conditions, it’s best to check one of the sources above.

History Length: 2

User: Tell me a joke.


**Why don’t skeletons fight each other?**  

They don’t have the guts! 😄

History Length: 4

User: What's the capital of France?


...

The capital of France is **Paris**.

## Conversation Flow & Key Insights

| Step | Assistant Action | Key Insight |
|------|------------------|-------------|
| 1 | **Apologized** for lacking live weather data. | Acknowledges limitation, sets tone for transparent assistance. |
| 2 | **Offered educational help**: explain typical regional weather patterns. | Provides context beyond raw data. |
| 3 | **Suggested reliable weather services** to check live data. | Guides user to trustworthy sources. |
| 4 | **Provided a quick guide on interpreting forecasts** when a location is supplied. | Empowers user to understand forecast nuances. |
| 5 | **Acknowledged user’s location mention** (“Paris is the capital of France”). | Recognizes user input, ready to tailor advice to Paris. |

### Take‑away Summary
- The assistant focused on transparency and empowerment rather than giving unavailable real‑time data.  
- It offered actionable steps: learning patterns, using trusted services, and interpreting forecasts.  
- The user’s mention of Paris signals readiness to provide Paris‑specific guidance when the assistant receives the location.


--- Advanced Hierarchical Summarization (after 3 interactions) ---

History Length: 1

Interaction 4 - User: Explain quantum computing simply.


## Quantum Computing – A Quick, Simple Overview

| Concept | Classical Computer | Quantum Computer |
|---------|--------------------|------------------|
| **Basic unit** | **Bit** – 0 or 1 | **Qubit** – can be 0, 1, or *both at once* |
| **State** | Deterministic | Probabilistic (superposition) |
| **Interaction** | Bits are independent | Qubits can be *entangled* – the state of one instantly affects another |
| **Operations** | Logical gates (AND, OR, NOT) | Quantum gates (Hadamard, CNOT, etc.) that rotate qubit states |
| **Speed‑up** | Linear scaling | Exponential scaling for certain problems (e.g., factoring, search) |

---

### 1. Bits vs. Qubits

| Classical Bit | Quantum Qubit |
|---------------|---------------|
| One of two definite states: **0** or **1** | A **superposition**: a weighted mix of 0 and 1 |
| Example: a light switch is either off or on | Example: a spinning coin is both heads and tails *until you look* |

**Key idea:** A qubit can encode *more information* because it exists in many states simultaneously.

---

### 2. Superposition

- Think of a qubit as a spinning wheel that can point to any angle.  
- While spinning, you can consider it to be in **all** positions at once.  
- Only when you measure it does it “collapse” to a single outcome (0 or 1).

---

### 3. Entanglement

- Two qubits can become linked so that the state of one instantly tells you about the other, no matter how far apart they are.  
- This correlation allows quantum computers to process complex relationships between data points in parallel.

---

### 4. Quantum Gates & Circuits

- Quantum gates are like tiny “twists” that change the qubit’s probabilities.  
- By chaining many gates, you create a **quantum circuit** that performs a computation.  
- The circuit’s final measurement gives you a classical answer, but the computation happened in a massively parallel “superposed” space.

---

### 5. Why It Matters

| Problem | Classical Time | Quantum Time (theoretically) |
|---------|----------------|------------------------------|
| Factor large numbers (RSA) | Exponential | Polynomial (Shor’s algorithm) |
| Search unsorted database | O(n) | O(√n) (Grover’s algorithm) |
| Simulate quantum systems | Hard | Natural fit |

> **Bottom line:** Quantum computers don’t replace classical computers for everything. They excel at specific tasks where quantum effects provide a speed‑up.

---

### 6. Current Reality

- **Hardware**: Small, noisy qubits (tens–hundreds) that require cryogenic temperatures.  
- **Noise**: Errors happen; researchers use **error correction** and **fault‑tolerant** designs.  
- **Applications**: Early prototypes are used for cryptography research, material science, and optimization problems.

---

### 7. Quick Analogy

| Classical | Quantum |
|-----------|---------|
| Reading a book page by page | Reading every page at once (but you only get a single page when you “look”) |

---

### TL;DR

- **Classical bits**: 0 or 1.  
- **Quantum qubits**: 0 + 1 (superposition) + instant correlation (entanglement).  
- **Gates** manipulate probabilities, not deterministic bits.  
- **Speed‑ups** for specific problems (factoring, search, simulation).  
- **Hardware** still experimental, but progress is rapid.

Feel free to ask about any part that sparks curiosity—whether it’s the math behind superposition or the practical challenges of building a quantum chip!

History Length: 3

Final History:
System: Hierarchical summary of previous conversation:
## ...
User: Explain quantum computing simply....
Assistant: ## Quantum Computing – A Quick, Simple Overview

|...


**Final advanced Task 2**

In [ ]:
import openai
import os
import json
import jsonschema
import re
from IPython.display import display, Markdown
import time

# Set up Groq API client
os.environ["GROQ_API_KEY"] = ""  
client = openai.OpenAI(
    base_url="https://api.groq.com/openai/v1",
    api_key=os.environ.get("GROQ_API_KEY")
)

MODEL = "openai/gpt-oss-20b"

# Refined JSON schema for PII extraction
PII_SCHEMA = {
    "type": "object",
    "properties": {
        "name": {
            "type": "string",
            "description": "Full name of the person (required, e.g., 'John Doe').",
            "minLength": 1,
            "maxLength": 100
        },
        "email": {
            "type": ["string", "null"],
            "description": "Email address (optional, e.g., 'john@example.com'). Use null if absent.",
            "pattern": "^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\\.[a-zA-Z]{2,}$",
            "maxLength": 100
        },
        "phone": {
            "type": ["string", "null"],
            "description": "Phone number (optional, e.g., '+1-555-123-4567'). Use null if absent.",
            "pattern": "^\\+?\\d[\\d\\s\\-\\(\\)]{9,}$",
            "maxLength": 20
        },
        "location": {
            "type": "string",
            "description": "Location or city (required, e.g., 'New York, NY').",
            "minLength": 1,
            "maxLength": 100
        },
        "age": {
            "type": ["integer", "null"],
            "description": "Age as integer (optional, 0-120). Use null if absent.",
            "minimum": 0,
            "maximum": 120
        }
    },
    "required": ["name", "location"],
    "additionalProperties": False
}

# Mask PII for display (post-extraction privacy)
def mask_pii_for_display(transcript):
    """Mask PII in transcript for display, preserving original for extraction."""
    masked = transcript
    masked = re.sub(r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b', '[EMAIL]', masked)
    masked = re.sub(r'(\+?\d{1,3}[-.\s]?)?\(?\d{3}\)?[-.\s]?\d{3}[-.\s]?\d{4}', '[PHONE]', masked)
    masked = re.sub(r'\b\d{1,3}\s*(years? old|age)\b', '[AGE]', masked)
    return masked

# Advanced PII extraction with robust retry
def extract_pii(chat_transcript, max_retries=3):
    """
    Extract PII using function calling with enhanced prompting and error handling.
    Returns structured JSON or None on failure.
    """
    function_definition = {
        "name": "extract_pii",
        "description": "Extract PII from chat transcript: name (required), location (required), email/phone/age (optional, null if absent).",
        "parameters": PII_SCHEMA
    }
    
    messages = [
        {"role": "system", "content": """You are an expert PII extractor. Follow these steps:
1. Identify explicit mentions of name, email, phone, location, age in the transcript.
2. Extract ONLY explicitly mentioned data; use null for absent fields (email, phone, age).
3. Ensure name/location are non-empty strings if mentioned.
4. Validate formats: email (user@domain.com), phone (+digits or digits with separators), age (0-120).
5. Return valid JSON matching the schema, with no extra fields or inferences."""},
        {"role": "user", "content": chat_transcript}  # Send original transcript
    ]
    
    for attempt in range(max_retries):
        try:
            response = client.chat.completions.create(
                model=MODEL,
                messages=messages,
                functions=[function_definition],
                function_call={"name": "extract_pii"},
                temperature=0.1,
                max_tokens=500
            )
            pii_data = json.loads(response.choices[0].message.function_call.arguments)
            return pii_data
        except (json.JSONDecodeError, AttributeError) as e:
            if attempt < max_retries - 1:
                time.sleep(2 ** attempt)  # Exponential backoff
                continue
            # Fallback: Try raw completion
            try:
                response = client.chat.completions.create(
                    model=MODEL,
                    messages=messages + [{"role": "system", "content": "Return JSON matching the schema."}],
                    temperature=0.1,
                    max_tokens=500
                )
                pii_data = json.loads(response.choices[0].message.content)
                return pii_data
            except Exception as e2:
                display(Markdown(f"**Retry Failed ({max_retries}x)**: {str(e)} (function call), {str(e2)} (fallback)"))
                return None
    
    return None

# Enhanced validation with detailed reporting
def validate_pii(pii_data):
    """Validate PII against schema, return detailed results."""
    try:
        jsonschema.validate(instance=pii_data, schema=PII_SCHEMA)
        return {"valid": True, "details": "All fields conform to schema (required present, types/patterns match, no extras)."}
    except jsonschema.exceptions.ValidationError as e:
        return {"valid": False, "details": {"error": str(e), "field": e.path[0] if e.path else "schema"}}

# Demonstration: Parse 4 sample chats
sample_chats = [
    {
        "id": 1,
        "transcript": "User: Hi, I'm John Doe, emailing from john.doe@example.com. My phone is +1-555-123-4567. I live in New York, NY, and I'm 28 years old."
    },
    {
        "id": 2,
        "transcript": "Assistant: Thanks for reaching out. User: Sure, my name is Jane Smith, email jane@work.com, phone 123-456-7890, and location is London, UK. Age not mentioned."
    },
    {
        "id": 3,
        "transcript": "User: Hello, this is Alice Johnson from Seattle, WA. No email or phone shared, and I won't tell my age."
    },
    {
        "id": 4,
        "transcript": "User: I'm Bob Wilson, 35 years old, from Toronto, Canada. Contact me at bob.wilson@company.org or +1-416-555-0123."
    }
]

print("=== Advanced Task 2: JSON Schema PII Extraction Demo (2025 Best Practices) ===")
for chat in sample_chats:
    print(f"\n**Sample Chat {chat['id']}**")
    display(Markdown(f"**Original Transcript**:\n{chat['transcript']}"))
    display(Markdown(f"**Masked Transcript (for display)**:\n{mask_pii_for_display(chat['transcript'])}"))
    
    # Extract PII
    pii_data = extract_pii(chat['transcript'])
    if pii_data:
        display(Markdown(f"**Extracted PII**:\n```json\n{json.dumps(pii_data, indent=2)}\n```"))
    
        # Validate
        validation = validate_pii(pii_data)
        display(Markdown(f"**Validation Results**:\n```json\n{json.dumps(validation, indent=2)}\n```"))
        status = "✅ Valid" if validation["valid"] else "❌ Invalid"
        display(Markdown(f"**Status**: {status}"))
    else:
        display(Markdown("**Status**: ❌ Extraction failed (check API key/rate limits)"))

=== Advanced Task 2: JSON Schema PII Extraction Demo (2025 Best Practices) ===

**Sample Chat 1**


**Original Transcript**:
User: Hi, I'm John Doe, emailing from john.doe@example.com. My phone is +1-555-123-4567. I live in New York, NY, and I'm 28 years old.

**Masked Transcript (for display)**:
User: Hi, I'm John Doe, emailing from [EMAIL]. My phone is [PHONE]. I live in New York, NY, and I'm [AGE].

**Extracted PII**:
```json
{
  "age": 28,
  "email": "john.doe@example.com",
  "location": "New York, NY",
  "name": "John Doe",
  "phone": "+1-555-123-4567"
}
```

**Validation Results**:
```json
{
  "valid": true,
  "details": "All fields conform to schema (required present, types/patterns match, no extras)."
}
```

...

**Original Transcript**:
User: I'm Bob Wilson, 35 years old, from Toronto, Canada. Contact me at bob.wilson@company.org or +1-416-555-0123.

**Masked Transcript (for display)**:
User: I'm Bob Wilson, [AGE], from Toronto, Canada. Contact me at [EMAIL] or [PHONE].

**Extracted PII**:
```json
{
  "age": 35,
  "email": "bob.wilson@company.org",
  "location": "Toronto, Canada",
  "name": "Bob Wilson",
  "phone": "+1-416-555-0123"
}
```

**Validation Results**:
```json
{
  "valid": true,
  "details": "All fields conform to schema (required present, types/patterns match, no extras)."
}
```

**Status**: ✅ Valid